In [5]:
import pandas as pd
import numpy as np
from statistics import median
from sklearn.preprocessing import OneHotEncoder
import sys
np.set_printoptions(threshold=sys.maxsize)
from apyori import apriori
import pyfpgrowth

pip install apyori --user

pip install pyfpgrowth --user

In [6]:
data = pd.read_csv("Avocado Dataset.csv")
data_trail = pd.read_csv("Trail.csv")

In [3]:
data.head(5)

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,27-12-2015,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,20-12-2015,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,13-12-2015,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,06-12-2015,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,29-11-2015,1.29,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


In [4]:
data_trail.head(5)

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,20-12-2015,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
1,13-12-2015,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
2,06-12-2015,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
3,22-11-2015,NaN,55979.78,1184.27,48067.99,43.61,6683.91,6556.47,127.44,0.0,conventional,2015,Albany
4,20-12-2015,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany


# Q1

In [5]:
data1 = data.drop(['Date', 'AveragePrice', 'Total Volume', 'Small Bags', 'Large Bags', 'XLarge Bags', 'year', 'region'], axis=1)
data1

,4046,4225,4770,Total Bags,type
0,1036.74,54454.85,48.16,8696.87,conventional
1,674.28,44638.81,58.33,9505.56,conventional
2,794.70,109149.67,130.50,8145.35,conventional
3,1132.00,71976.41,72.58,5811.16,conventional
4,941.48,43838.39,75.78,6183.95,conventional
...,...,...,...,...,...
18245,1191.70,3431.50,0.00,9264.84,organic
18246,1191.92,2452.79,727.94,9394.11,organic
18247,1527.63,2981.04,727.01,10969.54,organic
18248,2894.77,2356.13,224.53,12014.15,organic


In [6]:
data1 = data1.drop(data1[data1['type']!='organic'].index)
data1

,4046,4225,4770,Total Bags,type
9126,8.16,88.59,0.00,892.80,organic
9127,30.24,172.14,0.00,960.65,organic
9128,10.44,178.70,0.00,806.82,organic
9129,90.29,104.18,0.00,963.95,organic
9130,0.00,94.73,0.00,736.96,organic
...,...,...,...,...,...
18245,1191.70,3431.50,0.00,9264.84,organic
18246,1191.92,2452.79,727.94,9394.11,organic
18247,1527.63,2981.04,727.01,10969.54,organic
18248,2894.77,2356.13,224.53,12014.15,organic


# Q2

In [7]:
data_trail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202 entries, 0 to 201
Data columns (total 13 columns):
Date            202 non-null object
AveragePrice    177 non-null object
Total Volume    202 non-null float64
4046            202 non-null float64
4225            202 non-null float64
4770            202 non-null float64
Total Bags      202 non-null float64
Small Bags      202 non-null float64
Large Bags      202 non-null float64
XLarge Bags     202 non-null float64
type            202 non-null object
year            202 non-null int64
region          202 non-null object
dtypes: float64(8), int64(1), object(4)
memory usage: 20.6+ KB


In [8]:
len_before = len(data_trail)
print("Length before dropping: ", len_before)
duplicates = data_trail[data_trail.duplicated(keep='last')]
duplicates

Length before dropping:  202


,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,20-12-2015,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
1,13-12-2015,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
2,06-12-2015,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
3,22-11-2015,NaN,55979.78,1184.27,48067.99,43.61,6683.91,6556.47,127.44,0.0,conventional,2015,Albany
4,20-12-2015,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
6,06-12-2015,1.21,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,organic,2015,xxxx
7,29-11-2015,1.21,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,organic,2015,yyyy


In [9]:
# Dropping duplicates
data2 = data_trail.drop_duplicates(keep='last')
print("Length after dropping: ", len(data2))
data2

Length after dropping:  195


,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
5,13-12-2015,1.21,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,organic,2015,xxxx
8,22-11-2015,1.21,55979.78,1184.27,48067.99,43.61,6683.91,6556.47,127.44,0.0,organic,2015,yyyy
9,06-12-2015,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
10,22-11-2015,NaN,55979.78,1184.27,48067.99,43.61,6683.91,6556.47,127.44,0.0,conventional,2015,Albany
11,06-12-2015,1.21,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,organic,2015,xxxx
...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,20-09-2015,NaN,498640.23,4376.74,398673.48,418.46,95171.55,91612.66,3558.89,0.0,conventional,2015,Boston
198,13-09-2015,NaN,655682.95,5422.29,560792.23,353.77,89114.66,84843.55,4271.11,0.0,conventional,2015,Boston
199,06-09-2015,NaN,577774.74,4237.44,477867.83,496.62,95172.85,94558.41,614.44,0.0,conventional,2015,Boston
200,30-08-2015,NaN,526664.87,4177.03,438502.90,554.04,83430.90,83242.01,188.89,0.0,conventional,2015,Boston


In [10]:
data2['AveragePrice'].unique()

array(['1.21', '1.08', nan, '1.35', '0.93', '1.12', '1.11', '1.07',
       '0.99', '1.06', '1.17', '1.24', '1.22', '0.96', '1', '1.13',
       '1.09', '0.95', '0.97', '1.15', '1.01', '1.14', '1.1', '1.04',
       '1.03', '1.16', '1.05', 'na', 'NAN', '1.18', '1.2', '1.26', '1.19',
       '1.27', '1.29', '1.28', '1.25', '1.23', '0.71', '0.94', '0.98',
       '0.91', '1.02'], dtype=object)

In [11]:
data2['AveragePrice'] = data2['AveragePrice'].replace(to_replace=['nil', np.nan, 'NAN', 'na'], value=0)
data2['AveragePrice'].unique()

C:\Users\rammo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


array(['1.21', '1.08', 0, '1.35', '0.93', '1.12', '1.11', '1.07', '0.99',
       '1.06', '1.17', '1.24', '1.22', '0.96', '1', '1.13', '1.09',
       '0.95', '0.97', '1.15', '1.01', '1.14', '1.1', '1.04', '1.03',
       '1.16', '1.05', '1.18', '1.2', '1.26', '1.19', '1.27', '1.29',
       '1.28', '1.25', '1.23', '0.71', '0.94', '0.98', '0.91', '1.02'],
      dtype=object)

In [12]:
data2['AveragePrice'] = data2['AveragePrice'].astype('float')
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 195 entries, 5 to 201
Data columns (total 13 columns):
Date            195 non-null object
AveragePrice    195 non-null float64
Total Volume    195 non-null float64
4046            195 non-null float64
4225            195 non-null float64
4770            195 non-null float64
Total Bags      195 non-null float64
Small Bags      195 non-null float64
Large Bags      195 non-null float64
XLarge Bags     195 non-null float64
type            195 non-null object
year            195 non-null int64
region          195 non-null object
dtypes: float64(9), int64(1), object(3)
memory usage: 21.3+ KB


C:\Users\rammo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
data2['AveragePrice'] = data2['AveragePrice'].replace(to_replace=0, value=1.25)
data2['AveragePrice'].unique()

C:\Users\rammo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


array([1.21, 1.08, 1.25, 1.35, 0.93, 1.12, 1.11, 1.07, 0.99, 1.06, 1.17,
       1.24, 1.22, 0.96, 1.  , 1.13, 1.09, 0.95, 0.97, 1.15, 1.01, 1.14,
       1.1 , 1.04, 1.03, 1.16, 1.05, 1.18, 1.2 , 1.26, 1.19, 1.27, 1.29,
       1.28, 1.23, 0.71, 0.94, 0.98, 0.91, 1.02])

# Q3

In [14]:
data3 = data

In [15]:
def binary(year):
    if(year>=2016):
        return 1
    else:
        return 0


data3['year'] = data3['year'].apply(binary)
data3

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,27-12-2015,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,0,Albany
1,20-12-2015,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,0,Albany
2,13-12-2015,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,0,Albany
3,06-12-2015,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,0,Albany
4,29-11-2015,1.29,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,0,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18245,28-01-2018,1.71,13888.04,1191.70,3431.50,0.00,9264.84,8940.04,324.80,0.0,organic,1,WestTexNewMexico
18246,21-01-2018,1.87,13766.76,1191.92,2452.79,727.94,9394.11,9351.80,42.31,0.0,organic,1,WestTexNewMexico
18247,14-01-2018,1.93,16205.22,1527.63,2981.04,727.01,10969.54,10919.54,50.00,0.0,organic,1,WestTexNewMexico
18248,07-01-2018,1.62,17489.58,2894.77,2356.13,224.53,12014.15,11988.14,26.01,0.0,organic,1,WestTexNewMexico


In [16]:
data3['year'].unique()

array([0, 1], dtype=int64)

# Q4

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18250 entries, 0 to 18249
Data columns (total 13 columns):
Date            18250 non-null object
AveragePrice    18222 non-null object
Total Volume    18250 non-null float64
4046            18250 non-null float64
4225            18250 non-null float64
4770            18250 non-null float64
Total Bags      18250 non-null float64
Small Bags      18250 non-null float64
Large Bags      18250 non-null float64
XLarge Bags     18250 non-null float64
type            18250 non-null object
year            18250 non-null int64
region          18250 non-null object
dtypes: float64(8), int64(1), object(4)
memory usage: 1.8+ MB


In [19]:
categorical_variables = ['type', 'year', 'region']
data4 = data[categorical_variables]
data4

,type,year,region
0,conventional,2015,Albany
1,conventional,2015,Albany
2,conventional,2015,Albany
3,conventional,2015,Albany
4,conventional,2015,Albany
...,...,...,...
18245,organic,2018,WestTexNewMexico
18246,organic,2018,WestTexNewMexico
18247,organic,2018,WestTexNewMexico
18248,organic,2018,WestTexNewMexico


In [20]:
un_type = data4['type'].unique()
un_year = data4['year'].unique()
un_region = data4['region'].unique()
print(un_type)
print(un_year)
print(un_region)

['conventional' 'organic']
[2015 2016 2017 2018]
['Albany' 'Atlanta' 'BaltimoreWashington' 'Boise' 'Boston'
 'BuffaloRochester' 'California' 'Charlotte' 'Chicago' 'CincinnatiDayton'
 'Columbus' 'DallasFtWorth' 'Denver' 'Detroit' 'GrandRapids' 'GreatLakes'
 'HarrisburgScranton' 'HartfordSpringfield' 'Houston' 'Indianapolis'
 'Jacksonville' 'LasVegas' 'LosAngeles' 'Louisville' 'MiamiFtLauderdale'
 'Midsouth' 'Nashville' 'NewOrleansMobile' 'NewYork' 'Northeast'
 'NorthernNewEngland' 'Orlando' 'Philadelphia' 'PhoenixTucson'
 'Pittsburgh' 'Plains' 'Portland' 'RaleighGreensboro' 'RichmondNorfolk'
 'Roanoke' 'Sacramento' 'SanDiego' 'SanFrancisco' 'Seattle'
 'SouthCarolina' 'SouthCentral' 'Southeast' 'Spokane' 'StLouis' 'Syracuse'
 'Tampa' 'TotalUS' 'West' 'WestTexNewMexico']


In [21]:
def get_int_en(arr):
    ans = {}
    for i in range(len(arr)):
        ans[arr[i]] = i
    return ans

int_type = get_int_en(un_type)
int_year = get_int_en(un_year)
int_region = get_int_en(un_region)

# print(int_type, "\n", int_year, "\n", int_region)
data4['type'] = data4['type'].map(int_type)
data4['year'] = data4['year'].map(int_year)
data4['region'] = data4['region'].map(int_region)
data4

C:\Users\rammo\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\rammo\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\rammo\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

,type,year,region
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,0
4,0,0,0
...,...,...,...
18245,1,3,53
18246,1,3,53
18247,1,3,53
18248,1,3,53


# Q5

In [7]:
data5 = data
data5

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,27-12-2015,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,20-12-2015,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,13-12-2015,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,06-12-2015,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,29-11-2015,1.29,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18245,28-01-2018,1.71,13888.04,1191.70,3431.50,0.00,9264.84,8940.04,324.80,0.0,organic,2018,WestTexNewMexico
18246,21-01-2018,1.87,13766.76,1191.92,2452.79,727.94,9394.11,9351.80,42.31,0.0,organic,2018,WestTexNewMexico
18247,14-01-2018,1.93,16205.22,1527.63,2981.04,727.01,10969.54,10919.54,50.00,0.0,organic,2018,WestTexNewMexico
18248,07-01-2018,1.62,17489.58,2894.77,2356.13,224.53,12014.15,11988.14,26.01,0.0,organic,2018,WestTexNewMexico


In [9]:
len(data5['region'].unique())

54

In [10]:
region_encoder = OneHotEncoder()
region_reshaped = np.array(data5['region']).reshape(-1, 1)
region_values = region_encoder.fit_transform(region_reshaped)
print(region_values.toarray()[:5])

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0.]]


In [11]:
region_reshaped

array([['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Albany'],
       ['Atl

# Q6

In [24]:
data6 = data
print(data6['AveragePrice'].isnull().sum())
print(len(data6))

28
18250


In [25]:
len(data6['AveragePrice'].unique())

263

In [26]:
data6.drop(data6[(data6['AveragePrice']=='nil') | (data6['AveragePrice']==np.nan) | (data6['AveragePrice']=='na') | (data6['AveragePrice']=='NAN')].index, inplace=True)
data6

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,27-12-2015,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,20-12-2015,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,13-12-2015,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,06-12-2015,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,29-11-2015,1.29,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18245,28-01-2018,1.71,13888.04,1191.70,3431.50,0.00,9264.84,8940.04,324.80,0.0,organic,2018,WestTexNewMexico
18246,21-01-2018,1.87,13766.76,1191.92,2452.79,727.94,9394.11,9351.80,42.31,0.0,organic,2018,WestTexNewMexico
18247,14-01-2018,1.93,16205.22,1527.63,2981.04,727.01,10969.54,10919.54,50.00,0.0,organic,2018,WestTexNewMexico
18248,07-01-2018,1.62,17489.58,2894.77,2356.13,224.53,12014.15,11988.14,26.01,0.0,organic,2018,WestTexNewMexico


In [27]:
data6 = data6.dropna()
data6

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,27-12-2015,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,20-12-2015,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,13-12-2015,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,06-12-2015,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,29-11-2015,1.29,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18245,28-01-2018,1.71,13888.04,1191.70,3431.50,0.00,9264.84,8940.04,324.80,0.0,organic,2018,WestTexNewMexico
18246,21-01-2018,1.87,13766.76,1191.92,2452.79,727.94,9394.11,9351.80,42.31,0.0,organic,2018,WestTexNewMexico
18247,14-01-2018,1.93,16205.22,1527.63,2981.04,727.01,10969.54,10919.54,50.00,0.0,organic,2018,WestTexNewMexico
18248,07-01-2018,1.62,17489.58,2894.77,2356.13,224.53,12014.15,11988.14,26.01,0.0,organic,2018,WestTexNewMexico


In [28]:
len(data6['AveragePrice'].unique())

259

# Q7

In [29]:
data7 = data
data7.drop(['AveragePrice'], axis=1)

,Date,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,27-12-2015,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,20-12-2015,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,13-12-2015,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,06-12-2015,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,29-11-2015,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany
...,...,...,...,...,...,...,...,...,...,...,...,...
18245,28-01-2018,13888.04,1191.70,3431.50,0.00,9264.84,8940.04,324.80,0.0,organic,2018,WestTexNewMexico
18246,21-01-2018,13766.76,1191.92,2452.79,727.94,9394.11,9351.80,42.31,0.0,organic,2018,WestTexNewMexico
18247,14-01-2018,16205.22,1527.63,2981.04,727.01,10969.54,10919.54,50.00,0.0,organic,2018,WestTexNewMexico
18248,07-01-2018,17489.58,2894.77,2356.13,224.53,12014.15,11988.14,26.01,0.0,organic,2018,WestTexNewMexico


# Q8

In [30]:
data8 = data
data8.describe()

,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,year
count,1.823000e+04,1.823000e+04,1.823000e+04,1.823000e+04,1.823000e+04,1.823000e+04,1.823000e+04,18230.000000,18230.000000
mean,8.512783e+05,2.933024e+05,2.952957e+05,2.285285e+04,2.398254e+05,1.823226e+05,5.439307e+04,3109.664143,2016.149259
std,3.455275e+06,1.265615e+06,1.204720e+06,1.075174e+05,9.867378e+05,7.465548e+05,2.440871e+05,17701.828471,0.939758
min,8.456000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,2015.000000
25%,1.082719e+04,8.531000e+02,2.999282e+03,0.000000e+00,5.069523e+03,2.835905e+03,1.280175e+02,0.000000,2015.000000
50%,1.075441e+05,8.690590e+03,2.895790e+04,1.849950e+02,3.975249e+04,2.634000e+04,2.653825e+03,0.000000,2016.000000
75%,4.331048e+05,1.112380e+05,1.502475e+05,6.244020e+03,1.107878e+05,8.333783e+04,2.205494e+04,133.330000,2017.000000
max,6.250565e+07,2.274362e+07,2.047057e+07,2.546439e+06,1.937313e+07,1.338459e+07,5.719097e+06,551693.650000,2018.000000


In [31]:
# Most frequently occurring value under every attribute.
data8.mode()

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,18-03-2018,1.15,2038.99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,organic,2017.0,Atlanta
1,NaN,NaN,2858.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Boise
2,NaN,NaN,3288.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Boston
3,NaN,NaN,3529.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,California
4,NaN,NaN,3713.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Charlotte
5,NaN,NaN,4103.97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chicago
6,NaN,NaN,7223.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CincinnatiDayton
7,NaN,NaN,9465.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Columbus
8,NaN,NaN,13234.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DallasFtWorth
9,NaN,NaN,15896.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Denver


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18230 entries, 0 to 18249
Data columns (total 13 columns):
Date            18230 non-null object
AveragePrice    18202 non-null object
Total Volume    18230 non-null float64
4046            18230 non-null float64
4225            18230 non-null float64
4770            18230 non-null float64
Total Bags      18230 non-null float64
Small Bags      18230 non-null float64
Large Bags      18230 non-null float64
XLarge Bags     18230 non-null float64
type            18230 non-null object
year            18230 non-null int64
region          18230 non-null object
dtypes: float64(8), int64(1), object(4)
memory usage: 1.9+ MB


In [33]:
# Correlation matrix
data8.corr()

,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,year
Total Volume,1.000000,0.977866,0.974186,0.872204,0.963047,0.967238,0.880638,0.747152,0.016939
4046,0.977866,1.000000,0.926127,0.833402,0.920055,0.925280,0.838637,0.699366,0.003032
4225,0.974186,0.926127,1.000000,0.887853,0.905795,0.916037,0.810027,0.688815,-0.009752
4770,0.872204,0.833402,0.887853,1.000000,0.792317,0.802733,0.698478,0.679867,-0.036744
Total Bags,0.963047,0.920055,0.905795,0.792317,1.000000,0.994335,0.943008,0.804227,0.071334
Small Bags,0.967238,0.925280,0.916037,0.802733,0.994335,1.000000,0.902588,0.806841,0.063712
Large Bags,0.880638,0.838637,0.810027,0.698478,0.943008,0.902588,1.000000,0.710848,0.087642
XLarge Bags,0.747152,0.699366,0.688815,0.679867,0.804227,0.806841,0.710848,1.000000,0.080838
year,0.016939,0.003032,-0.009752,-0.036744,0.071334,0.063712,0.087642,0.080838,1.000000


In [34]:
# Skweness of every attribute
data8.skew()

Total Volume     9.003001
4046             8.643707
4225             8.938088
4770            10.154498
Total Bags       9.750980
Small Bags       9.535662
Large Bags       9.791406
XLarge Bags     13.133000
year             0.213762
dtype: float64

# Q9

In [35]:
lst = ['apple']*3 + ['orange']*2 + ['banana']*2
data9 = pd.Series(lst)
data9

0     apple
1     apple
2     apple
3    orange
4    orange
5    banana
6    banana
dtype: object

In [36]:
probs = data9.value_counts(normalize=True)
probs

apple     0.428571
banana    0.285714
orange    0.285714
dtype: float64

Gini index

In [37]:
gini_index = 1 - (np.sum(np.square(probs)))
gini_index

0.653061224489796

Entropy

In [38]:
entropy = -1 * np.sum(np.log2(probs) * probs)
entropy

1.5566567074628228

Information gain

In [39]:
info_gain = 1 - entropy
info_gain

-0.5566567074628228

Lower entropy => less impure => more IG 

Higher entropy => high impure => less IG

In [40]:
lst2 = ['apple', 'orange', 'banana', 'mango', 'blueberry', 'watermelon', 'pear']
data9_2 = pd.Series(lst2)
data9_2

0         apple
1        orange
2        banana
3         mango
4     blueberry
5    watermelon
6          pear
dtype: object

In [41]:
probs2 = data9_2.value_counts(normalize=True)
probs2

pear          0.142857
blueberry     0.142857
orange        0.142857
mango         0.142857
watermelon    0.142857
banana        0.142857
apple         0.142857
dtype: float64

In [42]:
# Gini index
gini_index2 = 1 - (np.sum(np.square(probs2)))
gini_index2

0.8571428571428572

In [43]:
# Entropy
entropy2 = -1 * np.sum(np.log2(probs2) * probs2)
entropy2

2.807354922057604

In [44]:
# Information gain
info_gain2 = 1 - entropy2
info_gain2

-1.8073549220576042

We can see that second data set has high entropy means high impure data set(because more variablilty is there in data)

# Q11

In [45]:
store_data = pd.read_csv('store_data.csv', header=None)
store_data.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
records = []
for i in range(0, 7501):
    records.append([str(store_data.values[i,j]) for j in range(0, 20)])

In [47]:
association_rules = apriori(records, min_support=0.0045, min_confidence=0.2, min_lift=3, min_length=2)
association_results = list(association_rules)

In [48]:
print(len(association_results))

48


In [49]:
print(association_results[0])

RelationRecord(items=frozenset({'light cream', 'chicken'}), support=0.004532728969470737, ordered_statistics=[OrderedStatistic(items_base=frozenset({'light cream'}), items_add=frozenset({'chicken'}), confidence=0.29059829059829057, lift=4.84395061728395)])


In [50]:
for item in association_results:

    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    print("Rule: " + items[0] + " -> " + items[1])

    #second index of the inner list
    print("Support: " + str(item[1]))

    #third index of the list located at 0th
    #of the third index of the inner list

    print("Confidence: " + str(item[2][0][2]))
    print("=====================================")

Rule: light cream -> chicken
Support: 0.004532728969470737
Confidence: 0.29059829059829057
Rule: escalope -> mushroom cream sauce
Support: 0.005732568990801226
Confidence: 0.3006993006993007
Rule: escalope -> pasta
Support: 0.005865884548726837
Confidence: 0.3728813559322034
Rule: herb & pepper -> ground beef
Support: 0.015997866951073192
Confidence: 0.3234501347708895
Rule: tomato sauce -> ground beef
Support: 0.005332622317024397
Confidence: 0.3773584905660377
Rule: whole wheat pasta -> olive oil
Support: 0.007998933475536596
Confidence: 0.2714932126696833
Rule: shrimp -> pasta
Support: 0.005065991201173177
Confidence: 0.3220338983050847
Rule: light cream -> nan
Support: 0.004532728969470737
Confidence: 0.29059829059829057
Rule: shrimp -> chocolate
Support: 0.005332622317024397
Confidence: 0.23255813953488375
Rule: cooking oil -> spaghetti
Support: 0.004799360085321957
Confidence: 0.5714285714285714
Rule: escalope -> nan
Support: 0.005732568990801226
Confidence: 0.3006993006993007
Ru

# Q12

In [51]:
data12 = pd.read_excel("transaction_data.xlsx")
data12

,Transaction ID,Items
0,T1,"Apple, Orange, Grape"
1,T2,"Apple, Orange"
2,T3,"Mango, Pineapple, Papaya"
3,T4,"Apple, Papaya"
4,T5,"Grape, Orange, Pineapple"


In [52]:
patterns = pyfpgrowth. find_frequent_patterns(data12['Items'], 10)
patterns

{('a', 'e', 'e'): 30,
 ('a', 'e', 'e', 'p'): 89,
 ('a', 'a', 'e', 'e'): 31,
 ('a', 'a', 'e', 'e', 'p'): 31,
 ('e', 'e', 'p'): 32,
 ('e', 'e', 'p', 'p'): 31,
 ('a', 'e'): 33,
 ('a', 'e', 'p'): 97,
 ('a', 'a', 'e'): 38,
 ('a', 'a', 'e', 'p'): 38,
 ('e', 'p'): 34,
 ('e', 'p', 'p'): 32,
 ('p', 'p'): 13,
 ('a', 'p', 'p'): 40,
 ('a', 'p'): 41,
 ('a', 'a', 'p'): 51,
 ('a', 'a', 'a'): 12,
 ('a', 'a', 'a', 'p'): 33,
 ('a', 'a', 'p', 'p'): 33}

In [53]:
rules = pyfpgrowth. generate_association_rules(patterns,0.8)
rules

{('a', 'e'): (('p',), 1.1515151515151516),
 ('a', 'p'): ((), 0.8048780487804879),
 ('e', 'p'): ((), 0.9411764705882353),
 ('a', 'e', 'e'): (('p',), 1.0333333333333334),
 ('a', 'e', 'p'): ((), 0.9175257731958762),
 ('e', 'e', 'p'): ((), 0.96875),
 ('a', 'a', 'e'): (('p',), 1.0),
 ('a', 'a', 'e', 'e'): (('p',), 1.0),
 ('a', 'a', 'e', 'p'): ((), 0.8157894736842105),
 ('p', 'p'): (('a',), 2.5384615384615383),
 ('e', 'p', 'p'): ((), 0.96875),
 ('a', 'a', 'a'): (('p',), 2.75),
 ('a', 'p', 'p'): ((), 0.825)}

In [55]:
data12 = pd.read_table('chess.dat', names=['Items'])
data12

,Items
0,1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 34 ...
1,1 3 5 7 9 12 13 15 17 19 21 23 25 27 29 31 34 ...
2,1 3 5 7 9 12 13 16 17 19 21 23 25 27 29 31 34 ...
3,1 3 5 7 9 11 13 15 17 20 21 23 25 27 29 31 34 ...
4,1 3 5 7 9 11 13 15 17 19 21 23 25 27 29 31 34 ...
...,...
3191,2 4 5 7 9 11 13 16 17 19 21 23 26 27 29 33 34 ...
3192,2 4 5 7 9 11 13 16 17 19 21 23 26 27 29 33 34 ...
3193,2 4 5 7 9 11 13 16 17 19 21 23 26 27 29 31 34 ...
3194,2 4 5 8 9 11 13 16 17 19 21 23 26 27 30 33 35 ...


In [58]:
patterns = pyfpgrowth. find_frequent_patterns(data12['Items'], 10000)
patterns

KeyboardInterrupt: 